In [66]:
# Required packages (check which are required)
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import networkx as nx
import pandas as pd
import numpy as np
import scipy as sp
import datetime as dt
import community
from shapely.geometry import Polygon


import importlib
import ddom

import random
import math

import cmocean as cmo

In [67]:
# Read the data from data_processing.ipynb
sa_calibration_data = pd.read_csv('../Data_Labour/calibration_data.csv')
employment_SSYK = pd.read_csv('../Data_Labour/occupational_employment.csv', sep = ',')
SSYK_shock = pd.read_csv('../Data_Labour/occupation_shock.csv', sep = ',', index_col = 0)
hours_data = pd.read_csv('../Data_Labour/hours_data.csv', sep = ',', index_col = 0)

G = nx.read_graphml('../Data_Labour/Occ_mob_sweden.graphml')

In [74]:
importlib.reload(ddom)
employment = employment_SSYK[['ssyk3', '2016']]
employment = {str(employment['ssyk3'].iloc[i]):employment['2016'].iloc[i] for i in range(len(employment))}
node_names = G.nodes()

# setup network
employed = {str(name):e for name,e in employment.items() if str(name) in node_names}
unemployed = {name:0 for name in node_names}
vacancies = {name:0 for name in node_names}

target_demand = {str(name):e for name,e in employment.items() if str(name) in node_names}
of_data = SSYK_shock.to_dict()['Computerisation Probability']
of_data = {str(code):prob for code, prob in of_data.items()}

attributes = {'employed':employed, 'unemployed':unemployed, 'vacancies':vacancies,
                'target_demand':target_demand, 'comp_prob':of_data}

# Parameters short calibration
#delta_u = 0.009
#delta_nu = 0.002
#gamma_u = 0.1
#gamma_nu = gamma_u
#timestep = 14

# Paramters long calibration (currently running)
a = 0.035
delta_u = 0.011
delta_nu = 0.005
gamma_u = 0.1
gamma_nu = gamma_u
timestep = 8

period = 10.25
k = 0.79
avg_hours_0 = hours_data.loc[2016,'average_hours/year']

output = {}

for shock_period in [30]:
    output[str(shock_period)] = ddom.deterministic_simulation(G, delta_u, delta_nu, gamma_u, gamma_nu, 
                                       timestep, period, attributes, shock_period = shock_period, k = k, 
                                        avg_hours_0 = avg_hours_0, 
                                       t_0 = shock_period/2, complete_network = True)

Simulation started at:  2020-05-16 21:07:31.070556
Simulation took:  0:25:20.756653


In [75]:
for period, out in output.items():
    vac_data = pd.DataFrame(out['vacancy_data'])
    u_data = pd.DataFrame(out['unemployment_data'])
    e_data = pd.DataFrame(out['employment_data'])
    td_data = pd.DataFrame(out['target_demand_data'])
    td_data.to_csv('../Data_Labour/simulation_output/c_td_simulation_'+period+'.csv', sep = ',', index = False)
    vac_data.to_csv('../Data_Labour/simulation_output/c_vac_simulation_'+period+'.csv', sep = ',', index = False)
    e_data.to_csv('../Data_Labour/simulation_output/c_emp_simulation_'+period+'.csv', sep = ',', index = False)
    u_data.to_csv('../Data_Labour/simulation_output/c_unemp_simulation_'+period+'.csv', sep = ',', index = False)

In [59]:
for period, out in output.items():
    vac_data = pd.DataFrame(out['vacancy_data'])
    u_data = pd.DataFrame(out['unemployment_data'])
    e_data = pd.DataFrame(out['employment_data'])
    td_data = pd.DataFrame(out['target_demand_data'])
    td_data.to_csv('../Data_Labour/simulation_output/short_robust_td_simulation_'+period+'.csv', sep = ',', index = False)
    vac_data.to_csv('../Data_Labour/simulation_output/short_robust_vac_simulation_'+period+'.csv', sep = ',', index = False)
    e_data.to_csv('../Data_Labour/simulation_output/short_robust_emp_simulation_'+period+'.csv', sep = ',', index = False)
    u_data.to_csv('../Data_Labour/simulation_output/short_robust_unemp_simulation_'+period+'.csv', sep = ',', index = False)

In [ ]:
employment = employment_SSYK[['ssyk3', '2016']]
employment = {str(employment['ssyk3'].iloc[i]):employment['2016'].iloc[i] for i in range(len(employment))}
node_names = G.nodes()

importlib.reload(ddom)


# setup network
employed = {str(name):e for name,e in employment.items() if str(name) in node_names}
unemployed = {name:0 for name in node_names}
vacancies = {name:[] for name in node_names}

target_demand = {str(name):e for name,e in employment.items() if str(name) in node_names}
of_data = SSYK_shock.to_dict()['Computerisation Probability']
of_data = {str(code):prob for code, prob in of_data.items()}

attributes = {'employed':employed, 'unemployed':unemployed, 'vacancies':vacancies,
                'target_demand':target_demand, 'comp_prob':of_data}

# Parameters short calibration
#delta_u = 0.009
#delta_nu = 0.002
#gamma_u = 0.1
#gamma_nu = gamma_u
#timestep = 14

# Paramters long calibration (currently running)
delta_u = 0.011
delta_nu = 0.005
gamma_u = 0.1
gamma_nu = gamma_u
timestep = 8

# General parameters
period = 10.25
shock_period = 30
k = 0.79
avg_hours_0 = hours_data.loc[2016,'average_hours/year']
t_0 = shock_period/2

vac_datalist = []
u_datalist = []
e_datalist = []
td_datalist = []
lt_u_datalist = []

for i in range(1,6):
    output_abm = ddom.simulation(G, delta_u, delta_nu, gamma_u, gamma_nu, timestep, 
                                 period, shock_period, k, avg_hours_0, t_0, attributes, complete_network = True)
    vac_data = pd.DataFrame(output_abm['vacancy_data'])
    u_data = pd.DataFrame(output_abm['unemployment_data'])
    e_data = pd.DataFrame(output_abm['employment_data'])
    td_data = pd.DataFrame(output_abm['target_demand_data'])
    vac_datalist.append(vac_data)
    u_datalist.append(u_data)
    e_datalist.append(e_data)
    td_datalist.append(td_data)
    vac_data.to_csv('../Data_Labour/simulation_output/c_vac_simulation_'+str(i)+'.csv', sep = ',', index = False)
    e_data.to_csv('../Data_Labour/simulation_output/c_emp_simulation_'+str(i)+'.csv', sep = ',', index = False)
    u_data.to_csv('../Data_Labour/simulation_output/c_unemp_simulation_'+str(i)+'.csv', sep = ',', index = False)
    
    

Simulation started at:  2020-05-16 22:52:00.070637
